# RANZCR - Simple X-Ray viewer

You can view any train image searching it by UID or view any random image based on criteria.

In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import json

df_train = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train.csv')
df_train.columns = ['UID','ETTA','ETTB','ETTN','NGTA','NGTB','NGTI','NGTN','CVCA','CVCB','CVCN','SGCP','PID']
df_annotations = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train_annotations.csv')
df_annotations.columns = ['UID','label','data']

def plot(r, annotations):
    img = cv2.imread('../input/ranzcr-clip-catheter-line-classification/train/' + r.UID + '.jpg')
    fig = px.imshow(img, height=800)
    if annotations.isnull().values.any() == False:
        for i, ann in annotations.iterrows():
            df = pd.DataFrame(json.loads(ann.data),columns=['x', 'y'])
            fig.add_trace(go.Scatter(x=df.x, y=df.y, name=ann.label))
    
    title = 'Labels: '
    for name, value in r.iteritems():
        if len(name) == 4 and value == 1:
            title += ' ' + name

    fig.update_layout(title_text=title, xaxis_title=r.UID,) 
    fig.show()

def xray(query):
    df = df_train.query(query).sample(1)
    an = df.join(df_annotations.set_index('UID'),how='left',on='UID')
    plot(df.iloc[0], an[['label','data']])

In [ ]:
# examples:
# xray('CVCA == 1') # -- view a random X-Ray with CVC abnormal
# xray("UID == '1.2.826.0.1.3680043.8.498.59757398491099579448057921213132792160'") # -- view a specific X-Ray by UID
# xray('SGCP == 1') # -- view a random X-Ray with Swan Ganz catheter
# xray('SGCP == 1 and CVCA == 1') # -- view a random X-Ray with Swan Ganz catheter and CVC abnormal


xray("UID == '1.2.826.0.1.3680043.8.498.59757398491099579448057921213132792160'")